In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import warnings

warnings.filterwarnings('ignore')

In [2]:
spark = (SparkSession
         .builder
         .getOrCreate()
        )

22/09/07 08:22:01 WARN Utils: Your hostname, andre-PC resolves to a loopback address: 127.0.1.1; using 192.168.100.55 instead (on interface eno1)
22/09/07 08:22:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/07 08:22:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
DATA_DIR='/home/data/phd/research_projects/fink_science/fink_science/cbpf_classifier/data/alerts/'
df = (spark
         .read
         .format('parquet')
         .load(DATA_DIR+'part-00000-c1d37f00-5125-44ed-95d3-169d6b7395c3-c000.snappy.parquet')
     )

22/09/07 08:22:04 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [4]:
# df.printSchema()

In [5]:
# df.columns

In [6]:
# df.select(df['diaSource']['diaSourceId']).show()

In [7]:
# df.select((df
#            .diaSource
#            .diaSourceId
#           )
#          ).show()

### pypsark DataFrame format (only showed features used by classifier)

```
root
 |-- alertId: long (nullable = true)
 |-- diaSource: struct (nullable = true)
 |    |-- midPointTai: double (nullable = true)
 |    |-- filterName: string (nullable = true)
 |    |-- psFlux: float (nullable = true)
 |    |-- psFluxErr: float (nullable = true)
 |-- prvDiaSources: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- midPointTai: double (nullable = true)
 |    |    |-- filterName: string (nullable = true)
 |    |    |-- psFlux: float (nullable = true)
 |    |    |-- psFluxErr: float (nullable = true)
 |-- prvDiaForcedSources: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- midPointTai: double (nullable = true)
 |    |    |-- filterName: string (nullable = true)
 |    |    |-- psFlux: float (nullable = true)
 |    |    |-- psFluxErr: float (nullable = true)
 |-- prvDiaNondetectionLimits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- midPointTai: double (nullable = true)
 |    |    |-- filterName: string (nullable = true)
 |-- diaObject: struct (nullable = true)
 |    |-- mwebv: float (nullable = true)
 |    |-- z_final: float (nullable = true)
 |    |-- z_final_err: float (nullable = true)
 |    |-- hostgal_zspec: float (nullable = true)
 |    |-- hostgal_zspec_err: float (nullable = true)
 |    |-- hostgal_zphot: float (nullable = true)
 |    |-- hostgal_zphot_err: float (nullable = true)
```

In [8]:
df_change = df.select(
        df.diaSource.midpointTai,
        df.diaSource.filterName,
        df.diaSource.psFlux,
        df.diaSource.psFluxErr,
        df.diaObject.mwebv,
        df.diaObject.z_final,
        df.diaObject.z_final_err,
        df.diaObject.hostgal_zphot,
        df.diaObject.hostgal_zphot_err)

In [9]:
df_data = (df_change
    .withColumnRenamed("diaSource.midpointTai", "midpointTai")
    .withColumnRenamed("diaSource.midpointTai", "midpointTai")
    .withColumnRenamed("diaSource.filterName", "filterName")
    .withColumnRenamed("diaSource.psFlux", "psFlux")
    .withColumnRenamed("diaSource.psFluxErr", "psFluxErr")
    .withColumnRenamed("diaObject.mwebv", "mwebv")
    .withColumnRenamed("diaObject.z_final", "z_final")
    .withColumnRenamed("diaObject.z_final_err", "z_final_err")
    .withColumnRenamed("diaObject.hostgal_zphot", "hostgal_zphot")
    .withColumnRenamed("diaObject.hostgal_zphot_err", "hostgal_zphot_err")
)

In [8]:
def concat_col(
        df, colname: str, prefix: str = 'c',
        current: str = 'candidate', history: str = 'prv_candidates'):
    """ Add new column to the DataFrame named `prefix`+`colname`, containing
    the concatenation of historical and current measurements.
    Parameters
    ----------
    df: DataFrame
        Pyspark DataFrame containing alert data
    colname: str
        Name of the column to add (without the prefix)
    prefix: str
        Additional prefix to add to the column
    """
    return df.withColumn(
            prefix + colname,
            F.when(
                df['{}.{}'.format(history, colname)].isNotNull(),
                F.concat(
                    df['{}.{}'.format(history, colname)],
                    F.array(df['{}.{}'.format(current, colname)])
                )
            ).otherwise(F.array(df['{}.{}'.format(current, colname)]))
        )
    

In [17]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 58.6 MB/s eta 0:00:00m eta 0:00:01:01:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 957.2/957.2 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.9 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.5 MB/s eta 0:00:00


In [22]:
what = ['midPointTai', 'psFlux', 'psFluxErr']
import numpy as np
import matplotlib.pyplot as plt

for element in what:
    df = concat_col(df, element, current='diaSource', history='prvDiaForcedSources')
    
#df.toPandas()['cmidPointTai'].values[2:4]):

for i, mjd in enumerate(np.array(df.select('cmidPointTai').collect())[2:4]):
    flux = np.array(df.select('cpsFlux').collect())[i][0]
    flux_err = np.array(df.select('cpsFluxErr').collect())[i][0]
    
    print(np.array(flux_err).shape,  np.array(mjd[0]).shape, np.array(flux).shape)
    #lc = np.vstack([mjd[0], flux, flux_err]).T
    
    
    
#plt.plot(lc[:, 0], lc[:,1])

(328,) (292,) (328,)


(64,) (78,) (64,)


In [18]:
df_new_column.show()

AttributeError: 'NoneType' object has no attribute 'show'

In [16]:
df_data.show()

+-----------+----------+---------+---------+-----------+-----------+-----------+-------------+-----------------+
|midpointTai|filterName|   psFlux|psFluxErr|      mwebv|    z_final|z_final_err|hostgal_zphot|hostgal_zphot_err|
+-----------+----------+---------+---------+-----------+-----------+-----------+-------------+-----------------+
| 61125.1297|         z| 32415.53|926.96313| 0.09092952|       -9.0|       -9.0|         -9.0|             -9.0|
| 61125.1456|         z|28024.938|535.88776| 0.06947285| 0.08547318|    0.02945|   0.08547318|          0.02945|
| 61125.1465|         z| 6534.219|1155.5437|0.046737716|       -9.0|       -9.0|         -9.0|             -9.0|
| 61125.2011|         i|494.15225|280.01733| 0.02286238| 0.12899986|    0.03063|   0.12899986|          0.03063|
| 61125.2301|         g|815.78754|109.06171| 0.06075122|  0.9675264|    0.05358|    0.9675264|          0.05358|
| 61125.2393|         g|6045.9805| 506.4182| 0.10580315|       -9.0|       -9.0|         -9.0|  

### Testing 

In [102]:
import numpy as np
import pandas as pd

from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType, IntegerType, StringType

def normalize_light_curve(lc_array: np.array) -> np.array:
    """
    Normalize light curves.

    Parameters:
    ----------
    lc_array: 1D np.array
        Input light curve of an alert.
        
    Returns:
    --------
    result: np.array
        normalized light curve of an alert.
    """

    result = np.zeros((lc_array.shape[0],))
    result[:] = lc_array[:]
    result[:] -= lc_array[0]

    norm = (lc_array[:] - np.min(lc_array[:])) / np.ptp(lc_array[:])
    result[:] = norm

    return result

@pandas_udf(StringType(), PandasUDFType.SCALAR)
def map_filter_name(filter_name: pd.Series) -> pd.Series:
    filter_dict = {'u':1, 'g':2, 'r':3, 'i':4, 'z':5, 'Y':6}
    if len(filter_name) < 10:
    
        filter_name = filter_name.copy()
        filter_name.map(filter_dict)
    
    else:
        nfilter_name = filter_name
        
    return pd.Series(filter_name)
    

In [103]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType

import tensorflow as tf
import numpy as np
import pandas as pd

@pandas_udf(DoubleType(), PandasUDFType.SCALAR)
def predict_nn(
        midpointTai: pd.Series, psFlux: pd.Series, psFluxErr: pd.Series,
        filterName: pd.Series,
        mwebv: pd.Series, z_final: pd.Series,
        z_final_err: pd.Series, hostgal_zphot: pd.Series,
        hostgal_zphot_err: pd.Series
    ) -> pd.Series:
    
    """
    Return predctions from a model given inputs as pd.Series

    Parameters:
    -----------
    midpointTai: spark DataFrame Column
        SNID JD Time (float)
    psFlux: spark DataFrame Column
        flux from LSST (float)
    psFluxErr: spark DataFrame Column
        flux error from LSST (float)
    filterName:
        (string)
    mwebv:
        (float)
    z_final: spark DataFrame Column
        redshift of a given event (float)
    z_final_err: spark DataFrame Column
        redshift error of a given event (float)       
    hostgal_zphot: spark DataFrame Column
        photometric redshift of host galaxy (float)
    hostgal_zphot_err: spark DataFrame Column
        error in photometric redshift of host galaxy (float)
    Returns:
    --------
    preds: pd.Series
        predictions of a broad class in an pd.Series format (pd.Series[float])
    """
    
    bands = []
    lcs = []
    meta = []

    filterName = apply_filter_name(filterName)
    #new_midpointTai = normalize_light_curve(midpointTai)
    
    
    return pd.Series(filterName)
    #return pd.Series(midpoinTai)

    for i, mjds in enumerate(midpointTai):
        print(i, mjds)
        bands.append(np.array(
             [filter_dict[f] for f in filterName[i]]
         ).astype(np.int16))
        
    return pd.Series(bands)
#         lc = np.concatenate(
            [mjds[:,None], psFlux[i][:,None], psFluxErr[:,None]], axis=-1
            )
        lcs.append(normalize_lc(lc).astype(np.float32))
        meta.append(
            np.concatenate(
                [mwebv[i], z_final[i], z_final_err[i], hostgal_zphot[i], hostgal_zphot_err[i]]
            )
        )

    X = {
        'meta': np.array(meta),
        'band': tf.RaggedTensor.from_row_lengths(
            values=tf.concat(bands, axis=0),
            row_lengths=[a.shape[0] for a in bands]
        ),

        'lc': tf.RaggedTensor.from_row_lengths(
            values=tf.concat(lcs, axis=0),
            row_lengths=[a.shape[0] for a in lcs]
        )
    }
    for i, x in enumerate(X['meta'][:,3]):
        if x < 0:
            X['meta'][i,1:] = -1
        else:
            X['meta'][i,1:] = x

    model='../models/model_test_meta_ragged_1det_after/'
    NN = tf.keras.models.load_model(model)
    preds = NN.predict(X)
    
    return pd.Series(preds)

In [104]:
import warnings
warnings.filterwarnings('ignore')

df_data.withColumn('mapFilterName',
                   predict_nn(
                       'midpointTai',
                       'filterName',
                       'psFlux',
                       'psFluxErr',
                       'mwebv',
                       'z_final',
                       'z_final_err',
                       'hostgal_zphot',
                       'hostgal_zphot_err',
                   )
                  ).show()

#df_data.withColumn('normalized', df_data['filterName']).show()

22/09/06 16:24:30 ERROR Executor: Exception in task 2.0 in stage 56.0 (TID 139)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/serializers.py", line 290, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 101, in <lambda>
    return lambda *a: (verify_result_length(*a), arrow_return_type)
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 92, in verify_result_length
  

Py4JJavaError: An error occurred while calling o1413.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 56.0 failed 1 times, most recent failure: Lost task 2.0 in stage 56.0 (TID 139, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/serializers.py", line 290, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/home/andsantos/.sdkman/candidates/spark/current/python/lib/pyspark.zip/pyspark/worker.py", line 101, in <lambda>
    return lambda *a: (v